# us-east-2

In [2]:
# specify the aws profile and region you want to use to deploy the stack
aws_profile="bbdev"
region_name="us-east-2"

MEGA = 1024*1024

In [3]:
lambda_name = !docker run \
    --rm \
    -v $HOME/.aws/credentials:/creds:ro \
    -v cloud-reader-tf:/mnt/state_vol \
    cloudfuse/cloud-reader-terraform output lambda_arn
lambda_name = lambda_name[0][1:len(lambda_name[0])-1]
print('lambda_name:', lambda_name)

lambda_name: arn:aws:lambda:us-east-2:615900053518:function:cloud-reader-benchmark


In [4]:
import boto3
import json
import base64

session = boto3.Session(profile_name=aws_profile)
client = session.client('lambda', region_name = region_name)

In [5]:
def bench_setup_1(max_parallel, ranges):
    inputParams = {
        "region": region_name,
        "bucket": "cloudfuse-taxi-data",
        "key": "tpch/tbl-s1/lineitem.tbl",
        "size": 759863287,
        "ranges": ranges,
        "max_parallel": max_parallel,
    }
    response = client.invoke(
        FunctionName = lambda_name,
        InvocationType = 'RequestResponse',
        Payload = json.dumps(inputParams),
        LogType='Tail'
    )
    print(base64.b64decode(response['LogResult']).decode("utf-8") )
    print(json.load(response['Payload']))


### 100MB chunks

In [32]:
bench_setup_1(32, [{"start": 0, "length": 100*MEGA}])

START RequestId: c2de5630-3de9-4b7e-838b-41ed68a9e933 Version: $LATEST
END RequestId: c2de5630-3de9-4b7e-838b-41ed68a9e933
REPORT RequestId: c2de5630-3de9-4b7e-838b-41ed68a9e933	Duration: 1285.74 ms	Billed Duration: 1321 ms	Memory Size: 3008 MB	Max Memory Used: 132 MB	Init Duration: 34.26 ms	

{'init_duration': 18, 'range_durations': [1262]}


In [33]:
bench_setup_1(32, [{"start": 0, "length": 100*MEGA}])

START RequestId: 323d3c03-1208-465b-9aa6-e98999b4f5ac Version: $LATEST
END RequestId: 323d3c03-1208-465b-9aa6-e98999b4f5ac
REPORT RequestId: 323d3c03-1208-465b-9aa6-e98999b4f5ac	Duration: 1116.33 ms	Billed Duration: 1117 ms	Memory Size: 3008 MB	Max Memory Used: 232 MB	

{'init_duration': 0, 'range_durations': [1115]}


### 10MB chunks

In [34]:
bench_setup_1(32, [{"start": 0, "length": 10*MEGA}])

START RequestId: 280fc413-e624-4c02-aded-4912dd0d60bb Version: $LATEST
END RequestId: 280fc413-e624-4c02-aded-4912dd0d60bb
REPORT RequestId: 280fc413-e624-4c02-aded-4912dd0d60bb	Duration: 112.30 ms	Billed Duration: 113 ms	Memory Size: 3008 MB	Max Memory Used: 242 MB	

{'init_duration': 0, 'range_durations': [111]}


In [35]:
bench_setup_1(32, [{"start": 0, "length": 10*MEGA}])

START RequestId: c6b360cf-4f1e-4a67-b0db-39ddd8b67a29 Version: $LATEST
END RequestId: c6b360cf-4f1e-4a67-b0db-39ddd8b67a29
REPORT RequestId: c6b360cf-4f1e-4a67-b0db-39ddd8b67a29	Duration: 108.39 ms	Billed Duration: 109 ms	Memory Size: 3008 MB	Max Memory Used: 252 MB	

{'init_duration': 0, 'range_durations': [107]}


In [10]:
bench_setup_1(32, [{"start": i*10, "length": 10*MEGA} for i in range(0,10)])

START RequestId: 7c9d043b-a5cb-4dfc-aa75-f418c9a2cb31 Version: $LATEST
END RequestId: 7c9d043b-a5cb-4dfc-aa75-f418c9a2cb31
REPORT RequestId: 7c9d043b-a5cb-4dfc-aa75-f418c9a2cb31	Duration: 714.70 ms	Billed Duration: 715 ms	Memory Size: 3008 MB	Max Memory Used: 136 MB	

{'init_duration': 1, 'range_durations': [640, 640, 640, 689, 689, 689, 708, 708, 708, 708]}


In [11]:
bench_setup_1(32, [{"start": i*10, "length": 10*MEGA} for i in range(0,20)])

START RequestId: c5dff323-9458-411a-a6dd-684540d57218 Version: $LATEST
END RequestId: c5dff323-9458-411a-a6dd-684540d57218
REPORT RequestId: c5dff323-9458-411a-a6dd-684540d57218	Duration: 1368.34 ms	Billed Duration: 1369 ms	Memory Size: 3008 MB	Max Memory Used: 237 MB	

{'init_duration': 1, 'range_durations': [1345, 1345, 1345, 1345, 1356, 1356, 1356, 1356, 1356, 1356, 1356, 1356, 1356, 1356, 1356, 1356, 1356, 1356, 1356, 1356]}


In [12]:
bench_setup_1(10, [{"start": i*10, "length": 10*MEGA} for i in range(0,20)])

START RequestId: 0e8d6cb2-4904-4fa5-92d2-cbbf83c19cee Version: $LATEST
END RequestId: 0e8d6cb2-4904-4fa5-92d2-cbbf83c19cee
REPORT RequestId: 0e8d6cb2-4904-4fa5-92d2-cbbf83c19cee	Duration: 871.85 ms	Billed Duration: 872 ms	Memory Size: 3008 MB	Max Memory Used: 237 MB	

{'init_duration': 1, 'range_durations': [637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 785, 817, 825, 858, 858, 861, 861, 861]}


### Notes:
- *Max Memory Used* keeps increasing execution after execution (until OOM)
- check that downloading 20 chunks is only 2x the duration of 10 chunks (with the proper `max_parallel`)

# eu-west-1

In [12]:
# specify the aws profile and region you want to use to deploy the stack
aws_profile="bbdev"
region_name="eu-west-1"

MEGA = 1024*1024

In [13]:
lambda_name = !docker run \
    --rm \
    -v $HOME/.aws/credentials:/creds:ro \
    -v cloud-reader-tf:/mnt/state_vol \
    cloudfuse/cloud-reader-terraform output lambda_arn
lambda_name = lambda_name[0][1:len(lambda_name[0])-1]
print('lambda_name:', lambda_name)

lambda_name: arn:aws:lambda:eu-west-1:615900053518:function:cloud-reader-benchmark


In [14]:
import boto3
import json
import base64

session = boto3.Session(profile_name=aws_profile)
client = session.client('lambda', region_name = region_name)

In [15]:
def bench_setup_1(ranges):
    inputParams = {
        "region": region_name,
        "bucket": "cloudfuse-taxi-data-eu",
        "key": "tpch/tbl-s1/lineitem.tbl",
        "size": 759863287,
        "ranges": ranges
    }
    response = client.invoke(
        FunctionName = lambda_name,
        InvocationType = 'RequestResponse',
        Payload = json.dumps(inputParams),
        LogType='Tail'
    )
    print(base64.b64decode(response['LogResult']).decode("utf-8") )
    print(json.load(response['Payload']))


In [16]:
bench_setup_1([{"start": 0, "length": 100*MEGA}])

START RequestId: 303a4bb6-bff4-46e0-8b98-7c2112e5bf0d Version: $LATEST
END RequestId: 303a4bb6-bff4-46e0-8b98-7c2112e5bf0d
REPORT RequestId: 303a4bb6-bff4-46e0-8b98-7c2112e5bf0d	Duration: 1098.28 ms	Billed Duration: 1134 ms	Memory Size: 3008 MB	Max Memory Used: 132 MB	Init Duration: 34.99 ms	

{'init_duration': 20, 'range_durations': [1072]}
